In [1]:
import numpy as np
from quaos.symplectic import PauliSum, PauliString, Pauli, Xnd, Ynd, Znd, Id, string_to_symplectic, symplectic_to_string
from quaos.gates import GateOperation, Circuit, Hadamard as H, SUM as CX, PHASE as S
from quaos.hamiltonian import *
from quaos.core.prime_Functions_Andrew import int_to_bases
from collections import defaultdict
import sympy as sym
from sympy.physics.quantum import TensorProduct,Operator
import itertools
from itertools import combinations

In [2]:
def xi(a, d):
    """
    Computes the a-th eigenvalue of a pauli with dimension d.
    
    Parameters:
        a (int): The integer to compute the eigenvalue for.
        d (int): The dimension of the pauli to use.
    
    Returns:
        complex: The computed eigenvalue.
    """
    return np.exp(2 * np.pi * 1j * a / d)

def group_indices(lst):
    """
    Groups indices of the same value in a list into sublists.
    For example, if the input list is [1, 2, 1, 3, 2], the output will be [[0, 2], [1, 4], [3]].
    """
    index_dict = defaultdict(list)
    for idx, value in enumerate(lst):
        index_dict[value].append(idx)
    
    return [indices for indices in index_dict.values()]


def Hadamard_Symmetric_PauliSum(n_paulis,n_qubits,n_sym_q):
    # create coefficients
    c_int_bands = np.sort(np.random.randint(n_paulis,size=n_paulis))
    c_bands = group_indices(c_int_bands)

    coefficients = np.zeros(n_paulis)
    sym_bands = []
    for i,b in enumerate(c_bands):
        coefficients[b] = np.random.normal(0, 1)
        if len(b) != 1:
            sym_bands.append(b)

    #print(sym_bands)
    n_extra_bands = np.floor(np.sum([len(b) for b in sym_bands])/2 - n_sym_q)
    pauli_strings = ['' for i in range(n_paulis)]

    all_x = []
    all_z = []
    for i in range(n_sym_q):
        x_pauli = []
        z_pauli = []
        if len(sym_bands) >= 1:
            b_ind = np.random.randint(len(sym_bands))
            b = sym_bands[b_ind]
            x_ind = np.random.randint(len(b))
            x_pauli.append(b[x_ind])
            b.pop(x_ind)
            z_ind = np.random.randint(len(b))
            z_pauli.append(b[z_ind])
            b.pop(z_ind)
            if len(b) < 2:
                sym_bands.pop(b_ind)
            else:
                sym_bands[b_ind] = b

            # randomly adding extra x and zs if possible
            if n_extra_bands > 0 and len(sym_bands) >= 1:
                extras = np.random.randint(n_extra_bands)
                n_extra_bands -= extras
                for j in range(extras):
                    b_ind = np.random.randint(len(sym_bands))
                    b = sym_bands[b_ind]
                    x_ind = np.random.randint(len(b))
                    x_pauli.append(b[x_ind])
                    b.pop(x_ind)
                    z_ind = np.random.randint(len(b))
                    z_pauli.append(b[z_ind])
                    b.pop(z_ind)
                    if len(b) < 2:
                        sym_bands.pop(b_ind)
                    else:
                        sym_bands[b_ind] = b
        print(coefficients[x_pauli])
        for j in range(n_paulis):
            if j in x_pauli:
                pauli_strings[j] += 'x1z0 '
            elif j in z_pauli:
                pauli_strings[j] += 'x0z1 '
            else:
                pauli_strings[j] += 'x0z0 '
        all_x += x_pauli
        all_z += z_pauli
    print(all_x,all_z)
    non_sym_paulis = [i for i in range(n_paulis) if not i in all_x and not i in all_z]
    q_dims = [2 for i in range(2*(n_qubits-n_sym_q))]
    available_paulis = list(np.arange(int(np.prod(q_dims))))
    for i,x in enumerate(all_x):
        pauli_index = np.random.choice(available_paulis)
        available_paulis.remove(pauli_index)
        exponents = int_to_bases(pauli_index, q_dims)
        for j in range(n_qubits-n_sym_q):
            r, s = int(exponents[2*j]), int(exponents[2*j+1])
            pauli_strings[x] += f"x{r}z{s} "
            pauli_strings[all_z[i]] += f"x{r}z{s} "

        pauli_strings[x].strip()
        pauli_strings[all_z[i]].strip()

    for i in non_sym_paulis:
        pauli_index = np.random.choice(available_paulis)
        available_paulis.remove(pauli_index)
        exponents = int_to_bases(pauli_index, q_dims)
        for j in range(n_qubits-n_sym_q):
            r, s = int(exponents[2*j]), int(exponents[2*j+1])
            pauli_strings[i] += f"x{r}z{s} "
        pauli_strings[i].strip()

    P = PauliSum(pauli_strings, weights=coefficients ,dimensions=[2 for i in range(n_qubits)], phases=None,standardise=False)
    #print(P)

    # construct random Clifford circuit
    C = Circuit(dimensions=[2 for i in range(n_qubits)])
    gate_list = [H,S,CX]
    gg = []
    for i in range(100):
        g_i = np.random.randint(3)
        if g_i == 2:
            aa = list(random.sample(range(n_qubits), 2))
            gg += [gate_list[g_i](aa[0],aa[1],2)]
            #print(aa)
        else:
            aa = list(random.sample(range(n_qubits), 1))
            gg += [gate_list[g_i](aa[0],2)]
        
    C.add_gate(gg)
    P = C.act(P)

    phases = P.phases
    cc = P.weights
    ss = P.pauli_strings
    dims = P.dimensions

    cc *= np.array([-1]*n_paulis)**phases
    P = PauliSum(ss, weights=cc ,dimensions=dims, phases=None,standardise=False)

    # qubits shuffle qubits (Fisher Yates Shuffle)
    '''
    gg = []
    order = np.arange(n_qubits)
    for i in np.arange(n_qubits)[::-1]:
        j = np.random.randint(i+1)
        gg += [GateOperation('SWAP',(i,j),['x1z0 x0z0 -> x0z0 x1z0', 'x0z1 x0z0 -> x0z0 x0z1', 'x0z0 x1z0 -> x1z0 x0z0', 'x0z0 x0z1 -> x0z1 x0z0'],2)]
        a = int(order[i])
        b = int(order[j])
        order[i] = b
        order[j] = a
        
    for g in gg:
        P = g.act(P)
    sym_qubit_ind = []
    for i in range(n_qubits):
        if order[i] in np.arange(n_sym_q):
            sym_qubit_ind.append(i)
    '''
    #print('Symmetric qubits: ',sym_qubit_ind)
    #print(P)
    return(P,C)   

def prepare_sym_candidates(P1,pi,pj,C,current_qubit,q_print=False):
    cq = current_qubit
    # prepare anti-commuting pauli strings with the same absolute coefficients for test of hadamard Symmetry
    # prime pauli pi and pj for cancel_pauli
    if q_print:
        print('pi',pi)
        print(P1.x_exp[pi, cq],P1.z_exp[pi, cq])
        print('pj',pj)
        print(P1.x_exp[pj, cq],P1.z_exp[pj, cq])

    if P1.x_exp[pi, cq] == 1 and P1.z_exp[pj, cq] == 1: # x,z

        px = pi
        pz = pj
    elif P1.z_exp[pi, cq] == 1 and P1.x_exp[pj, cq] == 1: # z,x
        px = pj
        pz = pi
    elif P1.x_exp[pi, cq] == 1 and P1.z_exp[pj, cq] == 0: # x,id or x,x
        if any(P1.z_exp[pj, i] for i in range(cq+1,P1.n_qudits())):
            g = CX(cq,min([i for i in range(cq+1,P1.n_qudits()) if P1.z_exp[pj, i]]),2)
        elif any(P1.x_exp[pj, i] for i in range(cq+1,P1.n_qudits())):
            g = H(min([i for i in range(cq+1,P1.n_qudits()) if P1.x_exp[pj, i]]),2)
            P1 = g.act(P1)
            C.add_gate(g)
            g = CX(cq,min([i for i in range(cq+1,P1.n_qudits()) if P1.z_exp[pj, i]]),2)
        C.add_gate(g)
        P1 = g.act(P1)
        px = pi
        pz = pj
    elif P1.z_exp[pi, cq] == 1 and P1.x_exp[pj, cq] == 0: # z,id or z,z
        if q_print:
            print('z,z','id,z')
        if any(P1.x_exp[pj, i] for i in range(cq+1,P1.n_qudits())):
            if q_print:
                print('x')
            g = CX(min([i for i in range(cq+1,P1.n_qudits()) if P1.x_exp[pj, i]]),cq,2)
        elif any(P1.z_exp[pj, i] for i in range(cq+1,P1.n_qudits())):
            if q_print:
                print('z')
                print(min([i for i in range(cq+1,P1.n_qudits()) if P1.z_exp[pj, i]]))
            g = H(min([i for i in range(cq+1,P1.n_qudits()) if P1.z_exp[pj, i]]),2)
            P1 = g.act(P1)
            C.add_gate(g)
            g = CX(min([i for i in range(cq+1,P1.n_qudits()) if P1.x_exp[pj, i]]),cq,2)
        C.add_gate(g)
        P1 = g.act(P1)
        if q_print:
            print(P1)
        px = pj
        pz = pi
    elif P1.x_exp[pi, cq] == 0 and P1.z_exp[pj, cq] == 1: # id,z
        if any(P1.x_exp[pi, i] for i in range(cq+1,P1.n_qudits())):
            g = CX(min([i for i in range(cq+1,P1.n_qudits()) if P1.x_exp[pi, i]]),cq,2)
        elif any(P.z_exp[pi, i] for i in range(cq+1,P1.n_qudits())):
            g = H(min([i for i in range(cq+1,P1.n_qudits()) if P1.z_exp[pi, i]]),2)
            P1 = g.act(P1)
            C.add_gate(g)
            g = CX(min([i for i in range(cq+1,P1.n_qudits()) if P1.x_exp[pi, i]]),cq,2)
        C.add_gate(g)
        P1 = g.act(P1)
        px = pi
        pz = pj
    elif P1.x_exp[pi, cq] == 0 and P1.x_exp[pj, cq] == 1: # id,x
        if any(P1.z_exp[pi, i] for i in range(cq+1,P1.n_qudits())):
            g = CX(cq,min([i for i in range(cq+1,P1.n_qudits()) if P1.z_exp[pi, i]]),2)
        elif any(P1.x_exp[pi, i] for i in range(cq+1,P1.n_qudits())):
            g = H(min([i for i in range(cq+1,P1.n_qudits()) if P1.x_exp[pi, i]]),2)
            P1 = g.act(P1)
            C.add_gate(g)
            g = CX(cq,min([i for i in range(cq+1,P1.n_qudits()) if P1.z_exp[pi, i]]),2)
        C.add_gate(g)
        P1 = g.act(P1)
        px = pj
        pz = pi
    else: # id,id
        if any(P1.x_exp[pi, i] for i in range(cq+1,P1.n_qudits())):
            g = CX(min([i for i in range(cq+1,P1.n_qudits()) if P1.x_exp[pi, i]]),cq,2)
            P1 = g.act(P1)
            C.add_gate(g)
            if any(P1.z_exp[pj, i] for i in range(cq+1,P1.n_qudits())):
                g = CX(cq,min([i for i in range(cq+1,P1.n_qudits()) if P1.z_exp[pj, i]]),2)
            elif any(P1.x_exp[pj, i] for i in range(cq+1,P1.n_qudits())):
                g = H(min([i for i in range(cq+1,P1.n_qudits()) if P1.x_exp[pj, i]]),2)
                P1 = g.act(P1)
                C.add_gate(g)
                g = CX(cq,min([i for i in range(cq+1,P1.n_qudits()) if P1.z_exp[pj, i]]),2)
            C.add_gate(g)
            P1 = g.act(P1)
            px = pi
            pz = pj
        elif any(P1.z_exp[pi, i] for i in range(cq+1,P1.n_qudits())):
            g = CX(cq,min([i for i in range(cq+1,P1.n_qudits()) if P1.z_exp[pi, i]]),2)
            P1 = g.act(P1)
            C.add_gate(g)
            if any(P1.x_exp[pj, i] for i in range(cq+1,P1.n_qudits())):
                g = CX(min([i for i in range(cq+1,P1.n_qudits()) if P1.x_exp[pj, i]]),cq,2)
            elif any(P1.z_exp[pj, i] for i in range(cq+1,P1.n_qudits())):
                g = H(min([i for i in range(cq+1,P1.n_qudits()) if P1.z_exp[pj, i]]),2)
                P1 = g.act(P1)
                C.add_gate(g)
                g = CX(min([i for i in range(cq+1,P1.n_qudits()) if P1.x_exp[pj, i]]),cq,2)
            C.add_gate(g)
            P1 = g.act(P1)
            px = pj
            pz = pi
    return(C, P1, px, pz)

def flatten_list(nested_list):
    """
    Flattens a list of lists into a single list with all the elements.
    
    Args:
        nested_list (list of lists): The input list of lists.
    
    Returns:
        list: A single flattened list containing all elements.
    """
    return [item for sublist in nested_list for item in sublist]

def check_current_paulis(P1,pi,pj,current_qubit,cc_bands):
    current_qubit_pauli_check = True
    for i in range(P1.n_paulis()):
        if i != pi and i != pj:
            if P1.x_exp[i, current_qubit] == 1 and P1.z_exp[i, current_qubit] == 1: # Y
                continue
            elif P1.x_exp[i, current_qubit] == 0 and P1.z_exp[i, current_qubit] == 0: # I
                continue
            elif (P1.x_exp[i, current_qubit] == 1 and P1.z_exp[i, current_qubit] == 0) or (P1.x_exp[i, current_qubit] == 0 and P1.z_exp[i, current_qubit] == 1): # X, Z
                if i not in flatten_list(cc_bands):
                    current_qubit_pauli_check = False
                    break
                else:
                    for b2 in cc_bands:
                        if i in b2:
                            if sum([P1.x_exp[j, current_qubit] for j in b2]) - sum([P1.z_exp[j, current_qubit] for j in b2]) == 0:
                                continue
                            else:
                                current_qubit_pauli_check = False
                                break
    return(current_qubit_pauli_check)

def hadamard_symmetry_pauli_candidates(pauli_sum,current_qubit):
    sym_paulis_candidates = [i for i in range(pauli_sum.n_paulis()) if (pauli_sum.x_exp[i,current_qubit] == 1 and pauli_sum.z_exp[i,current_qubit] == 0) or (pauli_sum.x_exp[i,current_qubit] == 0 and pauli_sum.z_exp[i,current_qubit] == 1)]
    other_paulis = [i for i in range(pauli_sum.n_paulis()) if i not in sym_paulis_candidates]
    return(sym_paulis_candidates,other_paulis)

def number_of_I(P,Pauli_index,qubits):
    """
    Returns the number of I's in the Pauli string at Pauli_index for qubits.
    """

    return np.sum([1 for i in qubits if P.x_exp[Pauli_index,i] == 0 and P.z_exp[Pauli_index,i] == 0])

def forcing_options(P,usable_qubits,sym_paulis_candidates):
    # For each PS, how many identities does it have on the still 'usable_qubits'
    list_n_I = [number_of_I(P,ip,usable_qubits) for ip in range(P.n_paulis())]
    
    # Locate Paulis that have all Identities (except for one) in the usable qubits and are not candidates for symmetry
    forcing_paulis = [i for i in range(P.n_paulis()) if list_n_I[i] == len(usable_qubits) - 1 and i not in sym_paulis_candidates]
    
    # locate the qubits on which the individual non-I paulis are
    forcing_qubits = []
    for i in forcing_paulis:
        qubit_index = min([j for j in usable_qubits if (P.x_exp[i,j]+P.z_exp[i,j]) != 0])
        if qubit_index not in forcing_qubits:
            forcing_qubits.append(qubit_index)
    
    return(forcing_paulis,forcing_qubits)

def reduce_gate_options(gate_options,gates_to_remove,qi,cq):
    for i in gates_to_remove:
        if i in gate_options[qi-(1+cq)]:
            gate_options[qi-(1+cq)].remove(i)
    return(gate_options)

def find_forced_gates_to_remove_Ys(P,cq,sym_paulis_candidates,other_paulis,q_print=False):

    h_sym = False # variable that becomes true once the PS is hadamard symmetric (afterwards we don't need to search for more forcings)
    no_remaining_forcings = False # variable that becomes true if there are no remaining forcings

    usable_qubits = [i for i in range(cq+1,P.n_qudits())]
    gate_options = [[0,1,2,3] for i in range(cq+1,P.n_qudits())]

    # x-component of the first qubit for all paulis (object that will be changed via forcings)
    all_first_qubit = P.x_exp[:,cq]

    while not no_remaining_forcings and len(usable_qubits) > 0 and not h_sym:
        forcing_paulis,forcing_qubits = forcing_options(P,usable_qubits,sym_paulis_candidates)

        no_remaining_forcings = True
        for qi in forcing_qubits:
            pauli_indexes = [i for i in forcing_paulis if (P.x_exp[i,qi]+P.z_exp[i,qi]) != 0]
            pauli_indexes_first_qubit = np.copy(all_first_qubit[pauli_indexes])
            
            # determine the number of different non-I paulis are on qubit qi for pauli_indexes (turn into own function)
            forcing_pauli_types = set([])
            for i in pauli_indexes:
                if P.x_exp[i,qi] == 1 and P.z_exp[i,qi] == 0:
                    forcing_pauli_types.add(1)
                elif P.x_exp[i,qi] == 0 and P.z_exp[i,qi] == 1:
                    forcing_pauli_types.add(2)
                elif P.x_exp[i,qi] == 1 and P.z_exp[i,qi] == 1:
                    forcing_pauli_types.add(3)
            if q_print:
                print('all_first_qubit',all_first_qubit)
                print('qi',qi)
                print('pauli_indexes',pauli_indexes)

            if len(forcing_pauli_types) > 1:
                if q_print:
                    print(pauli_indexes_first_qubit)
                    print((pauli_indexes_first_qubit + P.x_exp[pauli_indexes,qi])%2)
                    print((pauli_indexes_first_qubit + P.z_exp[pauli_indexes,qi])%2)
                    print((pauli_indexes_first_qubit + P.z_exp[pauli_indexes,qi] + P.x_exp[pauli_indexes,qi])%2)
                if not any(pauli_indexes_first_qubit):
                    # Identity
                    gate_options = reduce_gate_options(gate_options,[1,2,3],qi,cq)

                elif not any((pauli_indexes_first_qubit + P.x_exp[pauli_indexes,qi])%2):
                    # add_r2
                    gate_options = reduce_gate_options(gate_options,[0,2,3],qi,cq)
                    all_first_qubit = (all_first_qubit + P.x_exp[:,qi])%2
                    other_first_qubit = all_first_qubit[other_paulis]

                elif not any((pauli_indexes_first_qubit + P.z_exp[pauli_indexes,qi])%2):
                    # add_s2
                    gate_options = reduce_gate_options(gate_options,[0,1,3],qi,cq)
                    all_first_qubit = (all_first_qubit + P.z_exp[:,qi])%2

                elif not any((pauli_indexes_first_qubit + P.z_exp[pauli_indexes,qi] + P.x_exp[pauli_indexes,qi])%2):
                    # add_r2s2
                    gate_options = reduce_gate_options(gate_options,[0,1,2],qi,cq)
                    all_first_qubit = (all_first_qubit + P.x_exp[:,qi] + P.z_exp[:,qi])%2
                else:
                    # Forcing Impossible
                    gate_options = reduce_gate_options(gate_options,[0,1,2,3],qi,cq)
                    return(gate_options)
                no_remaining_forcings = False
                usable_qubits.remove(qi)
            else:
                '''
                removable_gates = []
                if any(pauli_indexes_first_qubit):
                    removable_gates.append(0)
                if any((pauli_indexes_first_qubit + P.x_exp[pauli_indexes,qi])%2):
                    removable_gates.append(1)
                if any((pauli_indexes_first_qubit + P.z_exp[pauli_indexes,qi])%2):
                    removable_gates.append(2)
                if any((pauli_indexes_first_qubit + P.z_exp[pauli_indexes,qi] + P.x_exp[pauli_indexes,qi])%2):
                    removable_gates.append(3) 
                gate_options = reduce_gate_options(gate_options,removable_gates,qi,cq)
                '''
                pass
            
            other_first_qubit = all_first_qubit[other_paulis]
            if not any(other_first_qubit%2):
                h_sym = True
                for iq in usable_qubits:
                    gate_options = reduce_gate_options(gate_options,[1,2,3],iq,cq)
    
    return(gate_options)

def search_gate_options(P,cq,gate_options,other_paulis,q_print=False):
    addition_options_lens = []
    for g in gate_options:
        addition_options_lens.append(len(g))
    max_options = np.prod(addition_options_lens)
    for i in range(max_options):
        addition_options_temp = int_to_bases(i,dims=addition_options_lens[::-1])[::-1]
        addition_options = np.array([gate_options[j][addition_options_temp[j]] for j in range(P.n_qudits() - cq - 1)])
        first_qubit = P.x_exp[other_paulis,cq]
        for k in range(P.n_qudits() - cq - 1):
            if addition_options[k] == 0:
                pass
            elif addition_options[k] == 1:
                first_qubit += P.x_exp[other_paulis,k+1+cq]
            elif addition_options[k] == 2:
                first_qubit += P.z_exp[other_paulis,k+1+cq]
            elif addition_options[k] == 3:
                first_qubit += P.x_exp[other_paulis,k+1+cq] + P.z_exp[other_paulis,k+1+cq]
        if q_print:
            print(addition_options)
            print(first_qubit%2)
        if not any(first_qubit%2):
            return(addition_options)
    # ToDo: Find a way to handle the case where no way to remove all Ys can be found
    return(addition_options)

def find_hadamard_sym_pairs(P,current_qubit,sym_paulis_candidates):
    sym_pairs = set([])
    for i,spc_i in enumerate(sym_paulis_candidates):
        for j,spc_j in enumerate(sym_paulis_candidates[i+1:]):
            if abs(P.weights[spc_i]) == abs(P.weights[spc_j]) and np.array_equal(P.x_exp[spc_i,current_qubit+1:],P.x_exp[spc_j,current_qubit+1:]) and np.array_equal(P.z_exp[spc_i,current_qubit+1:],P.z_exp[spc_j,current_qubit+1:]):
                sym_pairs.add((spc_i,spc_j))
    return(sym_pairs)

def calculate_pairs_parity(P,sym_pairs):
    pairs_parity = []
    for pairs in sym_pairs:
        if abs(P.weights[pairs[0]] * xi(P.phases[pairs[0]],2) - P.weights[pairs[1]] * xi(P.phases[pairs[1]],2)) < 10**-10:
            pairs_parity.append(0)
        else:
            pairs_parity.append(1)
    return(pairs_parity)

def parity_difference(P,cq,q2,sym_pairs,pairs_parity):
    parity_dict = {'I':set([]),'X':set([]),'Z':set([]),'Y':set([])}
    for ip,pairs in enumerate(sym_pairs):
        if P.x_exp[pairs[0],q2] == 0 and P.z_exp[pairs[0],q2] == 0:
            if pairs_parity[ip] == 0:
                parity_dict['I'].add(0)
            else:
                parity_dict['I'].add(1)
        elif P.x_exp[pairs[0],q2] == 1 and P.z_exp[pairs[0],q2] == 0:
            if pairs_parity[ip] == 0:
                parity_dict['X'].add(0)
            else:
                parity_dict['X'].add(1)
        elif P.x_exp[pairs[0],q2] == 0 and P.z_exp[pairs[0],q2] == 1:
            if pairs_parity[ip] == 0:
                parity_dict['Z'].add(0)
            else:
                parity_dict['Z'].add(1)
        elif P.x_exp[pairs[0],q2] == 1 and P.z_exp[pairs[0],q2] == 1:
            if pairs_parity[ip] == 0:
                parity_dict['Y'].add(0)
            else:
                parity_dict['Y'].add(1)
    return(parity_dict)

def update_phase_parity(pairs_parity,phase_options,P2,current_qubit,sym_pairs):
    pairs_parity_copy = pairs_parity.copy()
    for k in range(len(phase_options)):
        #print(k)
        if phase_options[k] == 0: # SSSS
            pass
        elif phase_options[k] == 1: # DDSS
            for ip,pairs in enumerate(sym_pairs):
                if P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                    pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                    pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                    pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                    pass
            #
        elif phase_options[k] == 2: # DSDS
            for ip,pairs in enumerate(sym_pairs):
                if P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                    pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    #pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                    #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                    pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    #pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                    #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    pass
            #
        elif phase_options[k] == 3: # DSSD
            for ip,pairs in enumerate(sym_pairs):
                if P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                    pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    #pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                    #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                    #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                    pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    #pass
            #
        elif phase_options[k] == 4: # SDDS
            for ip,pairs in enumerate(sym_pairs):
                if P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                    #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                    pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    #pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                    pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    #pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                    #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    pass
            #
        elif phase_options[k] == 5: # SDSD
            for ip,pairs in enumerate(sym_pairs):
                if P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                    #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                    pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    #pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                    #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                    pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    #pass
            #
        elif phase_options[k] == 6: # SSDD
            for ip,pairs in enumerate(sym_pairs):
                if P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                    #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                    #pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                    pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    #pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                    pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    #pass
            #
        elif phase_options[k] == 7: # DDDD
            for ip,pairs in enumerate(sym_pairs):
                if P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                    pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    #pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 0:
                    pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    #pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 0 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                    pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    #pass
                elif P2.x_exp[pairs[0],current_qubit + 1 + k] == 1 and P2.z_exp[pairs[0],current_qubit + 1 + k] == 1:
                    pairs_parity_copy[ip] = (pairs_parity_copy[ip] + 1)%2
                    #pass
    return(pairs_parity_copy)

def transform_list(lst):
    """
    Convert a list of 4 elements (0, 1, or 2) into a string where:
    - 0 → 'S'
    - 1 → 'D'
    - 2 → Either 'S' or 'D', chosen to ensure an even count of 'S' or 'D'
    
    Parameters:
        lst (list): A list of four elements containing 0s, 1s, and 2s.

    Returns:
        str: The resulting string with balanced 'S' and 'D'.
    """
    base_chars = ['S' if x == 0 else 'D' if x == 1 else None for x in lst]
    unknown_indices = [i for i, x in enumerate(lst) if x == 2]

    # Generate all possible assignments for '2's
    possible_values = ['S', 'D']
    for assignment in combinations(possible_values * len(unknown_indices), len(unknown_indices)):
        temp_chars = base_chars[:]
        for i, val in zip(unknown_indices, assignment):
            temp_chars[i] = val
        
        # Check if count of 'S' or 'D' is even
        if temp_chars.count('S') % 2 == 0 or temp_chars.count('D') % 2 == 0:
            return ''.join(temp_chars)
    
    return None  # In case no valid assignment is found (shouldn't happen)

def mode_to_int(mode):
    if mode == 'SSSS':
        return(0)
    elif mode == 'DDSS':
        return(1)
    elif mode == 'DSDS':
        return(2)
    elif mode == 'DSSD':
        return(3)
    elif mode == 'SDDS':
        return(4)
    elif mode == 'SDSD':
        return(5)
    elif mode == 'SSDD':
        return(6)
    elif mode == 'DDDD':
        return(7)
    else:
        raise ValueError("Mode must be one of SSSS, DDSS, DSDS, DSSD, SDDS, SDSD, SSDD, DDDD")

def find_phase_option(P,cq,sym_paulis_candidates):
    sym_pairs = find_hadamard_sym_pairs(P,cq,sym_paulis_candidates)
    pairs_parity = calculate_pairs_parity(P,sym_pairs)
    for n_gate in range(P.n_qudits() - cq - 1): # iterate by number of gates, not 'random' extensive search
        for comb in itertools.combinations(range(P.n_qudits() - cq - 1), n_gate): # choose n_gate gates to have a phase
            phase_options_lens = np.zeros(P.n_qudits() - cq - 1) + 1
            for i in comb:
                phase_options_lens[i] = 8
            max_options = int(np.prod(phase_options_lens))
            for i in range(max_options):
                pairs_parity_copy = pairs_parity.copy()
                phase_options = int_to_bases(i,dims=phase_options_lens)
                pairs_parity_copy = update_phase_parity(pairs_parity_copy,phase_options,P,cq,sym_pairs)
                if not any(pairs_parity_copy):
                    return(phase_options)
                else:
                    # check if any one gate option on one of the remaining qubits can fix the phases
                    remaining_qubits = [i + cq + 1  for i in range(P.n_qudits() - cq - 1) if phase_options_lens[i] == 1]
                    for q2 in remaining_qubits:
                        parity_dict = parity_difference(P,cq,q2,sym_pairs,pairs_parity_copy)
                        parity_dict_lens = [len(parity_option) for parity_option in parity_dict.values()]
                        if 2 not in parity_dict_lens:
                            parity_dict_vals = []
                            for parity_option in parity_dict.values():
                                if 0 in parity_option:
                                    parity_dict_vals.append(0)
                                elif 1 in parity_option:
                                    parity_dict_vals.append(1)
                                else:
                                    parity_dict_vals.append(2)
                            n_S = len([0 for _ in range(4) if parity_dict_vals[_] == 0])
                            n_D = len([1 for _ in range(4) if parity_dict_vals[_] == 1])
                            if n_S != 3 and n_D != 3:
                                mode = transform_list(parity_dict_vals)
                                phase_options[q2 - cq - 1] = mode_to_int(mode)
                                pairs_parity_copy = update_phase_parity(pairs_parity.copy(),phase_options,P,cq,sym_pairs)
                                if not any(pairs_parity_copy):
                                    return(phase_options)
                                else:
                                    print('Weird')
                
    # ToDo: Find a way to handle the case where no way to remove all Phases can be found
    return(phase_options)

def int_to_mode(n):
    if n == 0:
        return('SSSS')
    elif n == 1:
        return('DDSS')
    elif n == 2:
        return('DSDS')
    elif n == 3:
        return('DSSD')
    elif n == 4:
        return('SDDS')
    elif n == 5:
        return('SDSD')
    elif n == 6:
        return('SSDD')
    elif n == 7:
        return('DDDD')
    else:
        raise ValueError('Value has to be between 0 and 7')

def add_s2(P,q0,q1,phase_mode):
    if phase_mode == 'SSSS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())], 
                    gates=[CX(q0,q1,2),H(q1,2),S(q0,2),CX(q1,q0,2),H(q1,2),CX(q0,q1,2),S(q0,2)])
        return(C)
    elif phase_mode == 'DDSS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q0,q1,2),CX(q1,q0,2),H(q0,2),CX(q0,q1,2),H(q1,2),CX(q0,q1,2)])
        return(C)
    elif phase_mode == 'DSDS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q0,q1,2),H(q0,2),CX(q0,q1,2),CX(q1,q0,2),S(q0,2),CX(q1,q0,2),H(q1,2),CX(q0,q1,2),S(q0,2)])
        return(C)
    elif phase_mode == 'DSSD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q0,q1,2),H(q0,2),CX(q0,q1,2),S(q1,2),H(q1,2),CX(q1,q0,2),S(q0,2),CX(q1,q0,2),H(q1,2),CX(q0,q1,2),S(q0,2)])
        return(C)
    elif phase_mode == 'SDDS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q0,q1,2),H(q1,2),S(q1,2),CX(q1,q0,2),CX(q0,q1,2),S(q1,2),H(q1,2),S(q1,2),CX(q0,q1,2)])
        return(C)
    elif phase_mode == 'SDSD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q1,q0,2),S(q0,2),H(q1,2),CX(q0,q1,2),CX(q1,q0,2),S(q0,2)])
        return(C)    
    elif phase_mode == 'SSDD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q0,q1,2),H(q0,2),CX(q0,q1,2)])
        return(C) 
    elif phase_mode == 'DDDD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q0,q1,2),H(q0,2),S(q1,2),H(q1,2),S(q1,2),CX(q0,q1,2)])
        return(C) 

def add_r2(P,q0,q1,phase_mode):
    if phase_mode == 'SSSS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())], 
                    gates=[S(q0,2),CX(q1,q0,2),S(q0,2)])
        return(C)
    elif phase_mode == 'DDSS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q1,q0,2),H(q1,2),CX(q0,q1,2),CX(q1,q0,2),S(q0,2),CX(q1,q0,2),H(q1,2),CX(q0,q1,2),S(q0,2)])
        return(C)
    elif phase_mode == 'DSDS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[S(q0,2),CX(q1,q0,2),S(q0,2),H(q1,2),CX(q0,q1,2),S(q1,2),H(q1,2),S(q1,2),CX(q0,q1,2)])
        return(C)
    elif phase_mode == 'DSSD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q0,q1,2),S(q0,2),S(q1,2),CX(q0,q1,2),CX(q1,q0,2),S(q1,2),CX(q0,q1,2),S(q1,2),H(q1,2),S(q1,2),CX(q0,q1,2)])
        return(C)
    elif phase_mode == 'SDDS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[S(q0,2),CX(q1,q0,2),S(q0,2),S(q1,2),CX(q0,q1,2),S(q1,2),H(q1,2),S(q1,2),CX(q0,q1,2)])
        return(C)
    elif phase_mode == 'SDSD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q1,q0,2),H(q0,2),CX(q1,q0,2)])
        return(C)    
    elif phase_mode == 'SSDD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q1,q0,2),H(q0,2),CX(q1,q0,2),CX(q0,q1,2),S(1,2),H(q1,2),S(q1,2),CX(q0,q1,2)])
        return(C) 
    elif phase_mode == 'DDDD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q0,q1,2),S(q0,2),S(q1,2),CX(q0,q1,2),CX(q1,q0,2)])
        return(C)  

def add_r2s2(P,q0,q1,phase_mode):
    if phase_mode == 'SSSS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())], 
                    gates=[S(q0,2),CX(q1,q0,2),H(q1,2),CX(q1,q0,2),S(q0,2)])
        return(C)
    elif phase_mode == 'DDSS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[S(q0,2),CX(q1,q0,2),CX(q0,q1,2),S(q0,2),H(q0,2),CX(q0,q1,2)])
        return(C)
    elif phase_mode == 'DSDS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q0,q1,2),S(q0,2),S(q1,2),H(q0,2),CX(q1,q0,2),S(q1,2),H(q1,2),S(q1,2),CX(q0,q1,2)])
        return(C)
    elif phase_mode == 'DSSD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[S(q0,2),CX(q1,q0,2),H(q1,2),CX(q1,q0,2),S(q0,2),S(q1,2),CX(q0,q1,2),S(q1,2),H(q1,2),S(q1,2),CX(q0,q1,2)])
        return(C)
    elif phase_mode == 'SDDS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q0,q1,2),CX(q1,q0,2),S(q0,2),CX(q1,q0,2),H(q0,2),CX(q0,q1,2)])
        return(C)
    elif phase_mode == 'SDSD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q1,q0,2),CX(q0,q1,2),S(q0,2),CX(q1,q0,2),H(q0,2),CX(q0,q1,2),S(q0,2)])
        return(C)    
    elif phase_mode == 'SSDD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q0,q1,2),S(q0,2),S(q1,2),H(q0,2),CX(q1,q0,2),CX(q0,q1,2)])
        return(C) 
    elif phase_mode == 'DDDD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q0,q1,2),CX(q1,q0,2),S(q0,2),H(q0,2),H(q1,2),S(q1,2),CX(q0,q1,2),S(q0,2)])
        return(C) 

def add_phase(P,cq,qubit,phase_mode):
    if phase_mode == 'SSSS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())])
        return(C)
    elif phase_mode == 'DDSS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],gates=[CX(cq,qubit,2),S(qubit,2),H(qubit,2),S(qubit,2),CX(cq,qubit,2)])
        return(C)
    elif phase_mode == 'DSDS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],gates=[CX(cq,qubit,2),S(qubit,2),CX(cq,qubit,2),H(qubit,2),CX(cq,qubit,2),S(cq,2)])
        return(C)
    elif phase_mode == 'DSSD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],gates=[S(qubit,2),CX(cq,qubit,2),S(qubit,2),H(qubit,2),S(qubit,2),CX(cq,qubit,2)])
        return(C)
    elif phase_mode == 'SDDS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],gates=[S(qubit,2),H(qubit,2),CX(qubit,cq,2),S(cq,2),CX(qubit,cq,2),H(qubit,2),CX(cq,qubit,2),S(cq,2)])
        return(C)
    elif phase_mode == 'SDSD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],gates=[CX(qubit,cq,2),S(cq,2),CX(qubit,cq,2),H(qubit,2),CX(cq,qubit,2),S(cq,2)])
        return(C)    
    elif phase_mode == 'SSDD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],gates=[H(qubit,2),CX(qubit,cq,2),S(cq,2),CX(qubit,cq,2),H(qubit,2),CX(cq,qubit,2),S(cq,2)])
        return(C) 
    elif phase_mode == 'DDDD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],gates=[CX(qubit,cq,2),S(cq,2),CX(qubit,cq,2),H(qubit,2),CX(cq,qubit,2),S(cq,2),CX(cq,qubit,2),S(qubit,2),H(qubit,2),S(qubit,2),CX(cq,qubit,2)])
        return(C)

def addition_gate(i):
    if i == 0:
        return(add_phase)
    elif i == 1:
        return(add_r2)
    elif i == 2:
        return(add_s2)
    elif i == 3:
        return(add_r2s2)  

def get_remove_Y_circuit(P,cq,addition_options,phase_options):
    C = Circuit(dimensions=[2 for i in range(P.n_qudits())])
    for i,g in enumerate(addition_options):
        c = addition_gate(g)(P,cq,i+1+cq,int_to_mode(phase_options[i]))
        for ig in c.gates:
            C.add_gate(ig)
    return(C)

def remove_Ys(P1,C,current_qubit,q_print=False):
    cq = current_qubit
    sym_paulis_candidates,other_paulis = hadamard_symmetry_pauli_candidates(P1,current_qubit)

    # find forcings
    gate_options = find_forced_gates_to_remove_Ys(P1,cq,sym_paulis_candidates,other_paulis,q_print=q_print)
    max_gate_options = max([len(ig) for ig in gate_options])
    min_gate_options = min([len(ig) for ig in gate_options])
    if q_print:
        print('gate_options')
        print(gate_options)
        P2 = P1.copy()
        for ig,g in enumerate(gate_options):
            if len(g) == 1:
                f = addition_gate(g[0])
                c = f(P2,cq,ig+1+cq,'SSSS')
                print(c)
                P2 = c.act(P2)
        print('P2 according to current forcings')
        print(P2)

    
    # search through remaining gate options
    if min_gate_options == 0:
        return(P1,C)
    if max_gate_options > 1:
        addition_options = search_gate_options(P1,cq,gate_options,other_paulis,q_print=q_print)
    else:
        addition_options = np.array([g[0] for g in gate_options])

    # find phase options
    phase_options = find_phase_option(P1,cq,sym_paulis_candidates)
    
    # at this point we have addition_options which gives us which type of gate to use and phase_options which gives us which phase mode to use
    C1 = get_remove_Y_circuit(P1,cq,addition_options,phase_options)
    P1 = C1.act(P1)
    for g in C1.gates:
        C.add_gate(g)

    return(P1,C)


def PauliSum_Subspace(P,paulis,qubits):
    pauli_strings = P.pauli_strings
    weights = P.weights
    dimensions = P.dimensions
    phases = P.phases
    n_qudits = P.n_qudits()
    n_paulis = P.n_paulis()
    
    dimensions_new = np.array(dimensions)[qubits]
    pauli_strings_new = [PauliString(pauli_strings[i].x_exp[qubits],pauli_strings[i].z_exp[qubits],dimensions=dimensions_new) for i in paulis]
    weights_new = weights[paulis]
    phases_new = phases[paulis]
    P_new = PauliSum(pauli_strings_new, weights=weights_new ,dimensions=dimensions_new, phases=phases_new,standardise=False)
    return(P_new)

def Find_Hadamard_Symmetries(P,q_print=False):
    P1 = P.copy()
    C = Circuit(dimensions=[2 for i in range(P1.n_qudits())])
    # construct a list of all possible PS combinations that have the same absolute value coefficient 
    cc = P1.weights
    cc_abs = np.abs(cc)
    cc_bands = group_indices(cc_abs)
    cc_bands = [np.array(b) for b in cc_bands if len(b) > 1]
    current_qubit = 0
    for ib,b in enumerate(cc_bands):
        for ic,pi in enumerate(b):
            for jc,pj in enumerate(b[ic+1:]):
                P_temp = PauliSum_Subspace(C.act(P1),[pi,pj],[i for i in range(current_qubit,P1.n_qudits())])
                if q_print:
                    print(pi,pj)
                    print(P_temp)
                if not P_temp.is_commuting():
                    if q_print:
                        print('anticommuting')
                    C,current_qubit = Find_Hadamard_Symmetries_iter_(P1,C,pi,pj,current_qubit,cc_bands,q_print=q_print)
                    if q_print:
                        print(C.act(P1))
                        print()
    P1 = C.act(P1)
    return(P1,C,[i for i in range(current_qubit)])

def circuit_copy(C):
    C2 = Circuit(dimensions=C.dimensions)
    for g in C.gates:
        C2.add_gate(g)
    return(C2)

def Find_Hadamard_Symmetries_iter_(P,C,pi,pj,current_qubit,cc_bands,q_print=False):
    P1 = P.copy()
    P1 = C.act(P1)
    C1 = Circuit(dimensions=C.dimensions)
    
    P2 = P1.copy()
    C1, P2, px, pz = prepare_sym_candidates(P2,pi,pj,C1,current_qubit,q_print=q_print)
    if q_print:
        print('Original')
        print(P2)
        print()
        print('Circuit')
        print(C1.act(P1.copy()))

    # cancel for pauli with x
    P2, C1 = cancel_pauli(P2, current_qubit, px, C1, P2.n_qudits())
    # cancel for pauli with z
    g = H(current_qubit, P.dimensions[current_qubit])
    C1.add_gate(g)
    P2 = g.act(P2)
    P2, C1 = cancel_pauli(P2, current_qubit, pz, C1, P2.n_qudits())  
    g = H(current_qubit, P.dimensions[current_qubit])
    C1.add_gate(g)
    P2 = g.act(P2)
    if q_print:
        print('Make X111... and Z111...')
        print(P2)
        #print(C1.act(P1.copy()))

    current_qubit_pauli_check = check_current_paulis(P2,pi,pj,current_qubit,cc_bands)
    #print('current_qubit_pauli_check',current_qubit_pauli_check)
    if not current_qubit_pauli_check:
        # not a candidate for Hadamard symmetry, return just the original P1 and C
        return(C,current_qubit)

    for i in range(current_qubit+1,P1.n_qudits()):
        #print(i)
        C1, pivots = symplectic_reduction_iter_qudit_(P1.copy(), C1, [], i)
        #print(C1.act(P1.copy()))

    P2 = C1.act(P1.copy())
    if q_print:
        print('Symplectic Reduction')
        print(P2)

    # cancel the Y's in the first qubit
    #print(P2)
    P2,C1 = remove_Ys(P2,C1,current_qubit,q_print=q_print)
    if q_print:
        print('Remove Ys')    
        print(P2)
        print('Circuit')
        print(C1.act(P1.copy()))

    # check if all Y's in the first qubit have been cancelled
    sym_paulis_candidates,other_paulis = hadamard_symmetry_pauli_candidates(P2,current_qubit)
    first_qubit_x = P2.x_exp[other_paulis,current_qubit]
    if any(first_qubit_x%2):
        #print('No Hadamard Symmetry')
        return(C,current_qubit)
            
    # ToDo: final check for Hadamard symmetry
    # only x and z in the first qubit if both have the same coefficient
    # no problematic phases
    # after the x and z, all paulis are the same

    C2 = circuit_copy(C)
    for g in C1.gates:
        C2.add_gate(g)

    if q_print:
        print('Test')
        print(C1.act(C.act(P.copy())))
        print('Final Symmetrised')
        print(P2)
        print('Circuit')
        print(C2.act(P.copy()))

    return(C2,current_qubit+1)

def SWAP(P,q0,q1):
    C = Circuit(dimensions=[2 for i in range(P.n_qudits())],gates=[CX(q0,q1,2),CX(q1,q0,2),CX(q0,q1,2)])
    P = C.act(P)
    return(P)

def Hadamard_Symmetric_PauliSum(n_paulis,n_qubits,n_sym_q,q_print=False):
    # create coefficients
    c_int_bands = np.sort(np.random.randint(n_paulis,size=n_paulis))
    c_bands = group_indices(c_int_bands)

    coefficients = np.zeros(n_paulis)
    sym_bands = []
    for i,b in enumerate(c_bands):
        coefficients[b] = np.random.normal(0, 1)
        if len(b) != 1:
            sym_bands.append(b)

    #print(sym_bands)
    n_extra_bands = np.floor(np.sum([len(b) for b in sym_bands])/2 - n_sym_q)
    pauli_strings = ['' for i in range(n_paulis)]

    all_x = []
    all_z = []
    for i in range(n_sym_q):
        x_pauli = []
        z_pauli = []
        if len(sym_bands) >= 1:
            b_ind = np.random.randint(len(sym_bands))
            b = sym_bands[b_ind]
            x_ind = np.random.randint(len(b))
            x_pauli.append(b[x_ind])
            b.pop(x_ind)
            z_ind = np.random.randint(len(b))
            z_pauli.append(b[z_ind])
            b.pop(z_ind)
            if len(b) < 2:
                sym_bands.pop(b_ind)
            else:
                sym_bands[b_ind] = b

            # randomly adding extra x and zs if possible
            if n_extra_bands > 0 and len(sym_bands) >= 1:
                extras = np.random.randint(n_extra_bands)
                n_extra_bands -= extras
                for j in range(extras):
                    b_ind = np.random.randint(len(sym_bands))
                    b = sym_bands[b_ind]
                    x_ind = np.random.randint(len(b))
                    x_pauli.append(b[x_ind])
                    b.pop(x_ind)
                    z_ind = np.random.randint(len(b))
                    z_pauli.append(b[z_ind])
                    b.pop(z_ind)
                    if len(b) < 2:
                        sym_bands.pop(b_ind)
                    else:
                        sym_bands[b_ind] = b
        if q_print:
            print(coefficients[x_pauli])
        for j in range(n_paulis):
            if j in x_pauli:
                pauli_strings[j] += 'x1z0 '
            elif j in z_pauli:
                pauli_strings[j] += 'x0z1 '
            else:
                pauli_strings[j] += 'x0z0 '
        all_x += x_pauli
        all_z += z_pauli
    if q_print:
        print(all_x,all_z)
    non_sym_paulis = [i for i in range(n_paulis) if not i in all_x and not i in all_z]
    q_dims = [2 for i in range(2*(n_qubits-n_sym_q))]
    available_paulis = list(np.arange(int(np.prod(q_dims))))
    for i,x in enumerate(all_x):
        pauli_index = np.random.choice(available_paulis)
        available_paulis.remove(pauli_index)
        exponents = int_to_bases(pauli_index, q_dims)
        for j in range(n_qubits-n_sym_q):
            r, s = int(exponents[2*j]), int(exponents[2*j+1])
            pauli_strings[x] += f"x{r}z{s} "
            pauli_strings[all_z[i]] += f"x{r}z{s} "

        pauli_strings[x].strip()
        pauli_strings[all_z[i]].strip()

    for i in non_sym_paulis:
        pauli_index = np.random.choice(available_paulis)
        available_paulis.remove(pauli_index)
        exponents = int_to_bases(pauli_index, q_dims)
        for j in range(n_qubits-n_sym_q):
            r, s = int(exponents[2*j]), int(exponents[2*j+1])
            pauli_strings[i] += f"x{r}z{s} "
        pauli_strings[i].strip()

    P = PauliSum(pauli_strings, weights=coefficients ,dimensions=[2 for i in range(n_qubits)], phases=None,standardise=False)
    P_unchanged = P.copy()
    #print(P)

    sym_pauli_pairs_true,sym_q_true = PauliSum_hadamard_symmetry_check(P)
    if q_print:
        print(sym_pauli_pairs_true)
        print(sym_q_true)

    # construct random Clifford circuit
    C = Circuit(dimensions=[2 for i in range(n_qubits)])
    gate_list = [H,S,CX]
    gg = []
    for i in range(100):
        g_i = np.random.randint(3)
        if g_i == 2:
            aa = list(random.sample(range(n_qubits), 2))
            gg += [gate_list[g_i](aa[0],aa[1],2)]
            #print(aa)
        else:
            aa = list(random.sample(range(n_qubits), 1))
            gg += [gate_list[g_i](aa[0],2)]
        
    C.add_gate(gg)
    P = C.act(P)

    phases = P.phases
    cc = P.weights
    ss = P.pauli_strings
    dims = P.dimensions

    cc *= np.array([-1]*n_paulis)**phases
    P = PauliSum(ss, weights=cc ,dimensions=dims, phases=None,standardise=False)

    # qubits shuffle qubits 
    for i in range(10):
        a = np.random.randint(n_qubits)
        b = np.random.randint(n_qubits)
        if a != b:
            P = SWAP(P,a,b)
            
    
    #print(P)
    return(P,C,sym_pauli_pairs_true,len(sym_q_true),P_unchanged)


def hadamard_symmetry_check(P,q,sym_pauli_pairs,q_print=False):
    for i in range(P.n_paulis()):
        if q_print:
            print('Pauli',i)
        if P.x_exp[i,q] == 1 and P.z_exp[i,q] == 1:
            if q_print:
                print('Incompatible Pauli terms found:', i, P.x_exp[i,q], P.z_exp[i,q])
            return(False,sym_pauli_pairs)
        elif P.x_exp[i,q] == 0 and P.z_exp[i,q] == 0:
            if q_print:
                print('I')
            pass
        elif P.x_exp[i,q] == 1 and P.z_exp[i,q] == 0:
            if q_print:
                print('X')
            corresponding_z = False
            for j in range(P.n_paulis()):
                if q_print:
                    print('j',j)
                if P.x_exp[j,q] == 0 and P.z_exp[j,q] == 1:
                    if q_print:
                        print('z')
                    if np.abs(P.weights[i] - P.weights[j]) < 1e-10:
                        if q_print:
                            print('same weight')
                            print(P.x_exp[i,q+1:],P.x_exp[j,q+1:])
                            print(P.z_exp[i,q+1:],P.z_exp[j,q+1:])
                        if np.array_equal(P.x_exp[i,q+1:],P.x_exp[j,q+1:]) and np.array_equal(P.z_exp[i,q+1:],P.z_exp[j,q+1:]):
                            corresponding_z = True
                            sym_pauli_pairs.append({i,j})
                            break
            if not corresponding_z:
                return(False,sym_pauli_pairs)
        elif P.x_exp[i,q] == 0 and P.z_exp[i,q] == 1:
            if q_print:
                print('Z')
            corresponding_x = False
            for j in range(P.n_paulis()):
                if q_print:
                    print('j',j)
                    print(P.x_exp[j,q],P.z_exp[j,q])
                if P.x_exp[j,q] == 1 and P.z_exp[j,q] == 0:
                    if q_print:
                        print('x')
                    if np.abs(P.weights[i] - P.weights[j]) < 1e-10:
                        if q_print:
                            print('same weight')
                            print(P.x_exp[i,q+1:],P.x_exp[j,q+1:])
                            print(P.z_exp[i,q+1:],P.z_exp[j,q+1:])
                        if np.array_equal(P.x_exp[i,q+1:],P.x_exp[j,q+1:]) and np.array_equal(P.z_exp[i,q+1:],P.z_exp[j,q+1:]):
                            corresponding_x = True
                            break
            if not corresponding_x:
                return(False,sym_pauli_pairs)
        if q_print:
            print()
        
    return(True,sym_pauli_pairs)

def flatten_list_of_sets(L):
    new_list = []
    for s in L:
        for i in s:
            new_list.append(i)
    return new_list

def PauliSum_hadamard_symmetry_check(P,q_print=False):
    sym_check = True
    q = 0
    sym_pauli_pairs = []
    sym_q = []
    while sym_check == True:
        sym_check,sym_pauli_pairs = hadamard_symmetry_check(P,q,sym_pauli_pairs,q_print=q_print)
        if sym_check == True:
            sym_q.append(q)
        q += 1
    return(sym_pauli_pairs,sym_q)

In [ ]:
P,C,sym_pauli_pairs,n_sym_q,P_unchanged = Hadamard_Symmetric_PauliSum(97,5,1,q_print=False)
#P = SWAP(P,0,7)
print(P)
print(sym_pauli_pairs)
print(n_sym_q)

(-0.9905525909210074+0j) |x1z1 x0z1 x1z1 x0z1 x0z1 | 1 
(-0.9905525909210074+0j) |x1z1 x0z1 x1z1 x1z1 x0z1 | 1 
(0.9996617247598705-0j)  |x1z1 x0z1 x1z0 x1z1 x0z1 | 1 
(0.25756699948871864+0j) |x0z1 x0z1 x0z0 x0z0 x0z1 | 0 
(-0.31533817175949186+0j)|x1z1 x1z0 x1z1 x1z1 x1z1 | 0 
(-0.6389329140194798+0j) |x1z0 x1z1 x0z0 x0z0 x1z1 | 1 
(0.6389329140194798-0j)  |x1z0 x1z1 x0z1 x0z0 x1z1 | 1 
(1.7882684754288203-0j)  |x1z0 x1z0 x0z1 x1z0 x1z1 | 1 
(0.2640303037233088+0j)  |x1z0 x1z1 x1z0 x0z0 x0z0 | 0 
(0.2640303037233088+0j)  |x1z1 x1z1 x0z1 x1z1 x0z0 | 1 
(-0.2640303037233088+0j) |x0z1 x1z1 x1z0 x0z1 x1z0 | 0 
(0.2640303037233088+0j)  |x1z0 x0z0 x0z0 x0z1 x1z1 | 1 
(-0.8837146193356055+0j) |x0z1 x1z1 x0z1 x1z1 x0z1 | 0 
(-0.8382634371084642+0j) |x0z1 x1z1 x1z1 x0z0 x0z0 | 0 
(-1.4368153424873813+0j) |x1z0 x1z0 x1z0 x0z0 x0z0 | 0 
(1.7048388562264714-0j)  |x1z0 x1z0 x1z1 x1z1 x1z0 | 0 
(1.7048388562264714-0j)  |x1z1 x1z0 x0z0 x1z1 x0z0 | 1 
(1.0187501664507492+0j)  |x0z0 x1z1 x0z0 x1z1 x0

In [222]:
# check if the symmetries are correct for a lot of paulis
for i in range(100):
    print(i)
    n_paulis = np.random.randint(6,100)
    n_qubits = np.random.randint(3,10)
    number_of_sym = np.random.randint(0,n_qubits-2)
    print(n_paulis,n_qubits,number_of_sym)
    P,C,sym_pauli_pairs_true,n_sym_q_true,P_unchanged = Hadamard_Symmetric_PauliSum(n_paulis,n_qubits,number_of_sym)

    pauli_sum,C,sym_qubits = Find_Hadamard_Symmetries(P,q_print=False)
    pauli_sum.phase_to_weight()
    pauli_sum.weights = np.around(pauli_sum.weights,decimals=10)
    sym_pauli_pairs_calc,sym_q_calc = PauliSum_hadamard_symmetry_check(pauli_sum)
    if n_sym_q_true != len(sym_q_calc):
        print('Number of symmetries not equal')
        print(n_sym_q_true,len(sym_q_calc))
        break

    if not np.array_equal(np.sort(np.array(flatten_list_of_sets(sym_pauli_pairs_true))), np.sort(np.array(flatten_list_of_sets(sym_pauli_pairs_calc)))):
        print('Symmetries not equal')
        print(sym_pauli_pairs_true)
        print(sym_pauli_pairs_calc)
        break


    
    

0
54 4 1
1
15 4 1
2
17 8 0
3
15 6 3
4
40 8 0
5
43 4 0
6
44 6 1
7
9 8 4
Number of symmetries not equal
4 2


In [227]:
print(P)
print(np.sort(np.array(flatten_list_of_sets(sym_pauli_pairs_true))))
print(P_unchanged)

(-0.74418639509782+0j)    |x0z0 x1z0 x0z1 x0z0 x1z1 x0z1 x0z1 x1z1 | 1 
(0.74418639509782-0j)     |x0z1 x0z0 x0z1 x1z0 x1z1 x1z1 x1z1 x1z0 | 0 
(1.0566862049220846-0j)   |x0z1 x1z1 x1z0 x0z1 x1z0 x1z1 x1z0 x1z1 | 1 
(1.0566862049220846-0j)   |x1z0 x0z0 x1z0 x0z1 x0z1 x1z0 x0z0 x0z0 | 0 
(-1.7120402587343664+0j)  |x0z0 x1z0 x1z1 x1z1 x1z1 x1z0 x1z1 x0z1 | 0 
(-0.036889258011722874+0j)|x1z0 x0z0 x1z1 x0z1 x0z0 x0z1 x0z1 x0z0 | 1 
(-0.36891721440903763+0j) |x1z0 x0z0 x0z0 x1z1 x1z1 x1z1 x0z0 x1z1 | 0 
(0.36891721440903763+0j)  |x0z0 x0z1 x0z0 x0z1 x1z1 x0z0 x0z1 x1z1 | 1 
(0.5179244400931522+0j)   |x0z1 x1z1 x0z1 x1z0 x0z1 x1z0 x1z1 x0z0 | 0 

[0 1 2 3 6 7]
(-0.74418639509782+0j)    |x0z0 x1z0 x0z0 x0z0 x0z0 x1z1 x0z1 x0z0 | 0 
(-0.74418639509782+0j)    |x0z0 x0z1 x0z0 x0z0 x0z0 x1z1 x0z1 x0z0 | 0 
(-1.0566862049220846+0j)  |x0z0 x0z0 x0z1 x0z0 x1z0 x0z1 x0z0 x0z0 | 0 
(-1.0566862049220846+0j)  |x0z0 x0z0 x1z0 x0z0 x1z0 x0z1 x0z0 x0z0 | 0 
(-1.7120402587343664+0j)  |x0z0 x0z0 x0z0 x0z0 x1

In [225]:
pauli_sum,C,sym_qubits = Find_Hadamard_Symmetries(P,q_print=False)
pauli_sum.phase_to_weight()
pauli_sum.weights = np.around(pauli_sum.weights,decimals=10)
print(pauli_sum)
sym_pauli_pairs,sym_q = PauliSum_hadamard_symmetry_check(pauli_sum,q_print=False)
print(np.sort(np.array(flatten_list_of_sets(sym_pauli_pairs))))
print(sym_q)

(0.7441863951-0j) |x1z0 x0z0 x1z0 x0z0 x0z0 x0z0 x0z0 x0z0 | 0 
(0.7441863951+0j) |x0z1 x0z0 x1z0 x0z0 x0z0 x0z0 x0z0 x0z0 | 0 
(1.0566862049+0j) |x0z0 x0z1 x0z1 x1z0 x0z0 x0z0 x0z0 x0z0 | 0 
(1.0566862049+0j) |x0z0 x1z0 x0z1 x1z0 x0z0 x0z0 x0z0 x0z0 | 0 
(1.7120402587-0j) |x0z0 x0z0 x1z0 x1z0 x0z0 x0z0 x0z0 x0z0 | 0 
(0.036889258-0j)  |x0z0 x0z0 x0z1 x0z0 x0z0 x0z0 x0z0 x0z0 | 0 
(0.3689172144-0j) |x0z0 x0z0 x0z1 x0z1 x0z0 x0z0 x0z0 x0z0 | 0 
(-0.3689172144+0j)|x0z0 x0z0 x0z1 x1z1 x1z0 x0z0 x0z0 x0z0 | 0 
(0.5179244401+0j) |x0z0 x0z0 x1z0 x1z0 x1z0 x0z0 x0z0 x0z0 | 0 

[0 1 2 3]
[0, 1]


In [246]:
cc = P.weights
cc_abs = np.abs(cc)
cc_bands = group_indices(cc_abs)
cc_bands = [np.array(b) for b in cc_bands if len(b) > 1]
C = Circuit(dimensions=[2 for i in range(P.n_qudits())])
print(P)
print()
C,current_qubit = Find_Hadamard_Symmetries_iter_(P,C,6,7,0,cc_bands,q_print=False)

C,current_qubit = Find_Hadamard_Symmetries_iter_(P,C,2,3,current_qubit,cc_bands,q_print=True)

#C,current_qubit = Find_Hadamard_Symmetries_iter_(P,C,0,1,current_qubit,cc_bands,q_print=False)

#C,current_qubit = Find_Hadamard_Symmetries_iter_(P,C,22,23,current_qubit,cc_bands,q_print=False)
#P1,C = Find_Hadamard_Symmetries(P)
print(C.act(P))
#print(pauli_sum == C.act(P))

(-0.74418639509782+0j)    |x0z0 x1z0 x0z1 x0z0 x1z1 x0z1 x0z1 x1z1 | 1 
(0.74418639509782-0j)     |x0z1 x0z0 x0z1 x1z0 x1z1 x1z1 x1z1 x1z0 | 0 
(1.0566862049220846-0j)   |x0z1 x1z1 x1z0 x0z1 x1z0 x1z1 x1z0 x1z1 | 1 
(1.0566862049220846-0j)   |x1z0 x0z0 x1z0 x0z1 x0z1 x1z0 x0z0 x0z0 | 0 
(-1.7120402587343664+0j)  |x0z0 x1z0 x1z1 x1z1 x1z1 x1z0 x1z1 x0z1 | 0 
(-0.036889258011722874+0j)|x1z0 x0z0 x1z1 x0z1 x0z0 x0z1 x0z1 x0z0 | 1 
(-0.36891721440903763+0j) |x1z0 x0z0 x0z0 x1z1 x1z1 x1z1 x0z0 x1z1 | 0 
(0.36891721440903763+0j)  |x0z0 x0z1 x0z0 x0z1 x1z1 x0z0 x0z1 x1z1 | 1 
(0.5179244400931522+0j)   |x0z1 x1z1 x0z1 x1z0 x0z1 x1z0 x1z1 x0z0 | 0 


pi 2
1.0 1.0
pj 3
1.0 1.0
Original
(-0.74418639509782+0j)    |x0z0 x0z1 x0z0 x0z0 x0z0 x0z0 x0z0 x0z0 | 0 
(0.74418639509782-0j)     |x0z0 x1z0 x0z0 x0z0 x0z0 x0z0 x0z0 x0z0 | 0 
(1.0566862049220846-0j)   |x0z0 x1z1 x0z1 x0z0 x0z0 x0z0 x0z0 x0z0 | 0 
(1.0566862049220846-0j)   |x0z0 x1z1 x1z0 x0z0 x0z0 x0z0 x0z0 x0z0 | 0 
(-1.7120402587343664+0j)  |

In [7]:
def Hadamard_Symmetric_PauliSum(n_paulis,n_qubits,n_sym_q,q_print=False):
    # create coefficients
    c_int_bands = np.sort(np.random.randint(n_paulis,size=n_paulis))
    c_bands = group_indices(c_int_bands)

    coefficients = np.zeros(n_paulis)
    sym_bands = []
    for i,b in enumerate(c_bands):
        coefficients[b] = np.random.normal(0, 1)
        if len(b) != 1:
            sym_bands.append(b)

    #print(sym_bands)
    n_extra_bands = np.floor(np.sum([len(b) for b in sym_bands])/2 - n_sym_q)
    pauli_strings = ['' for i in range(n_paulis)]

    all_x = []
    all_z = []
    for i in range(n_sym_q):
        x_pauli = []
        z_pauli = []
        if len(sym_bands) >= 1:
            b_ind = np.random.randint(len(sym_bands))
            b = sym_bands[b_ind]
            x_ind = np.random.randint(len(b))
            x_pauli.append(b[x_ind])
            b.pop(x_ind)
            z_ind = np.random.randint(len(b))
            z_pauli.append(b[z_ind])
            b.pop(z_ind)
            if len(b) < 2:
                sym_bands.pop(b_ind)
            else:
                sym_bands[b_ind] = b

            # randomly adding extra x and zs if possible
            if n_extra_bands > 0 and len(sym_bands) >= 1:
                extras = np.random.randint(n_extra_bands)
                n_extra_bands -= extras
                for j in range(extras):
                    b_ind = np.random.randint(len(sym_bands))
                    b = sym_bands[b_ind]
                    x_ind = np.random.randint(len(b))
                    x_pauli.append(b[x_ind])
                    b.pop(x_ind)
                    z_ind = np.random.randint(len(b))
                    z_pauli.append(b[z_ind])
                    b.pop(z_ind)
                    if len(b) < 2:
                        sym_bands.pop(b_ind)
                    else:
                        sym_bands[b_ind] = b
        if q_print:
            print(coefficients[x_pauli])
        for j in range(n_paulis):
            if j in x_pauli:
                pauli_strings[j] += 'x1z0 '
            elif j in z_pauli:
                pauli_strings[j] += 'x0z1 '
            else:
                pauli_strings[j] += 'x0z0 '
        all_x += x_pauli
        all_z += z_pauli
    if q_print:
        print(all_x,all_z)
    non_sym_paulis = [i for i in range(n_paulis) if not i in all_x and not i in all_z]
    q_dims = [2 for i in range(2*(n_qubits-n_sym_q))]
    available_paulis = list(np.arange(int(np.prod(q_dims))))
    for i,x in enumerate(all_x):
        pauli_index = np.random.choice(available_paulis)
        available_paulis.remove(pauli_index)
        exponents = int_to_bases(pauli_index, q_dims)
        for j in range(n_qubits-n_sym_q):
            r, s = int(exponents[2*j]), int(exponents[2*j+1])
            pauli_strings[x] += f"x{r}z{s} "
            pauli_strings[all_z[i]] += f"x{r}z{s} "

        pauli_strings[x].strip()
        pauli_strings[all_z[i]].strip()

    for i in non_sym_paulis:
        pauli_index = np.random.choice(available_paulis)
        available_paulis.remove(pauli_index)
        exponents = int_to_bases(pauli_index, q_dims)
        for j in range(n_qubits-n_sym_q):
            r, s = int(exponents[2*j]), int(exponents[2*j+1])
            pauli_strings[i] += f"x{r}z{s} "
        pauli_strings[i].strip()

    P = PauliSum(pauli_strings, weights=coefficients ,dimensions=[2 for i in range(n_qubits)], phases=None,standardise=False)
    print(P)
    sym_pauli_pairs_true,sym_q_true = PauliSum_hadamard_symmetry_check(P)
    print(sym_q_true)
    symplectic_reduction_qudits = [i for i in range(P.n_qudits()) if i not in sym_q_true]

    d = P.dimensions
    q = P.n_qudits()
    P1 = P.copy()
    C = Circuit(d)
    pivots = []

    for i in symplectic_reduction_qudits:
        C, pivots = symplectic_reduction_iter_qudit_(P1.copy(), C, pivots, i)
    P1 = C.act(P1)

    removable_qubits = set([i for i in range(P.n_qudits()) if i not in sym_q_true]) - set([pivot[1] for pivot in pivots])
    conditional_qubits = sorted(set([i for i in range(P.n_qudits()) if i not in sym_q_true]) - removable_qubits - set([pivot[1] for pivot in pivots if pivot[2] == 'Z']))
    removable_qubits = sorted(list(removable_qubits))
    if any(conditional_qubits):

        for cq in conditional_qubits:
            g = H(cq, d[cq])
            C.add_gate(g)
        P1 = g.act(P1)


    print('removable_qubits',removable_qubits)
    print('conditional_qubits',conditional_qubits)
    print(removable_qubits + conditional_qubits)
    print(P1)
    print(P1.x_exp)
    qudits_to_remove = np.copy(sorted(removable_qubits + conditional_qubits)[::-1])
    P1.delete_qudits_(qudits_to_remove)
    print(P1)
    print(P1.pauli_strings)
    print(P1.weights)
    print(P1.dimensions)
    print(P1.phases)
    print(P1.x_exp)
    print(P1.z_exp)
    ps_copy = P1.pauli_strings.copy()
    print(ps_copy)
    print(P1.weights.copy())
    print(P1.phases.copy())
    print(P1.dimensions.copy())
    P2 = PauliSum(P1.pauli_strings.copy(), P1.weights.copy(), P1.phases.copy(), P1.dimensions.copy(), False)
    P_unchanged = P1.copy()
    #print(P)

    sym_pauli_pairs_true,sym_q_true = PauliSum_hadamard_symmetry_check(P)
    if q_print:
        print(sym_pauli_pairs_true)
        print(sym_q_true)

    # construct random Clifford circuit
    C = Circuit(dimensions=[2 for i in range(n_qubits)])
    gate_list = [H,S,CX]
    gg = []
    for i in range(100):
        g_i = np.random.randint(3)
        if g_i == 2:
            aa = list(random.sample(range(n_qubits), 2))
            gg += [gate_list[g_i](aa[0],aa[1],2)]
            #print(aa)
        else:
            aa = list(random.sample(range(n_qubits), 1))
            gg += [gate_list[g_i](aa[0],2)]
        
    C.add_gate(gg)
    P = C.act(P)

    phases = P.phases
    cc = P.weights
    ss = P.pauli_strings
    dims = P.dimensions

    cc *= np.array([-1]*n_paulis)**phases
    P = PauliSum(ss, weights=cc ,dimensions=dims, phases=None,standardise=False)

    # qubits shuffle qubits 
    for i in range(10):
        a = np.random.randint(n_qubits)
        b = np.random.randint(n_qubits)
        if a != b:
            P = SWAP(P,a,b)
            
    
    #print(P)
    return(P,C,sym_pauli_pairs_true,len(sym_q_true),P_unchanged)

In [8]:
Hadamard_Symmetric_PauliSum(8,10,3,q_print=False)

(-0.6084073035373349+0j)  |x1z0 x0z0 x0z0 x0z0 x0z0 x1z0 x0z0 x1z0 x0z1 x0z0 | 0 
(-0.6084073035373349+0j)  |x0z1 x0z0 x0z0 x0z0 x0z0 x1z0 x0z0 x1z0 x0z1 x0z0 | 0 
(1.8218253676123297+0j)   |x0z0 x0z0 x0z0 x0z0 x1z0 x1z1 x1z1 x1z0 x0z0 x1z0 | 0 
(-0.011133724246158138+0j)|x0z0 x0z0 x0z0 x0z1 x1z1 x0z1 x1z1 x1z0 x0z1 x1z1 | 0 
(-1.059318790790374+0j)   |x0z0 x0z0 x1z0 x1z0 x0z1 x1z0 x0z1 x0z0 x1z0 x0z1 | 0 
(-1.059318790790374+0j)   |x0z0 x0z0 x0z1 x1z0 x0z1 x1z0 x0z1 x0z0 x1z0 x0z1 | 0 
(1.0151718116030017+0j)   |x0z0 x1z0 x0z0 x0z1 x1z0 x0z1 x0z0 x1z1 x0z1 x1z1 | 0 
(1.0151718116030017+0j)   |x0z0 x0z1 x0z0 x0z1 x1z0 x0z1 x0z0 x1z1 x0z1 x1z1 | 0 

[0, 1, 2]
removable_qubits [5, 7, 8, 9]
conditional_qubits [6]
[5, 7, 8, 9, 6]
(-0.6084073035373349+0j)  |x1z0 x0z0 x0z0 x0z1 x0z0 x0z0 x0z0 x0z0 x0z0 x0z0 | 0 
(-0.6084073035373349+0j)  |x0z1 x0z0 x0z0 x0z1 x0z0 x0z0 x0z0 x0z0 x0z0 x0z0 | 0 
(1.8218253676123297+0j)   |x0z0 x0z0 x0z0 x1z0 x1z0 x0z0 x0z0 x0z0 x0z0 x0z0 | 0 
(-0.01113372424615

ValueError: operands could not be broadcast together with shapes (8,) (5,) 